### jupyter notebook to convert text to ids

author: Jan Jörg
date: 14.03.2024

In [1]:
# imports
import pandas as pd
from db import get_database

In [2]:
dbname = get_database()
collection = dbname["joined"]

documents = collection.find()
df = pd.DataFrame(list(documents))

df.head(2)

,_id,onetsoc19,display,occupation,tasks,technology_skills,tools_used,tools_technology,knowledge,skills,...,job_zone,interests,work_styles,work_values,related_occupations,additional_information,education,2010soc,isco08,Name_de
0,661d1a30d05f1cef398e0e97,27-2011.00,short,"{'code': '27-2011.00', 'title': 'Actors', 'tag...","{'task': [{'id': 7646, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.7.c', 'related': 'htt...","{'element': [{'id': '2.A.1.a', 'related': 'htt...",...,"{'value': 2, 'title': 'Job Zone Two: Some Prep...","{'high_point_code': 'ASE', 'element': [{'id': ...","{'element': [{'id': '1.C.3.a', 'related': 'htt...","{'element': [{'id': '1.B.2.d', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'http://www.actorsequity.o...,{'level_required': {'category': [{'name': 'Les...,27-2011,2655,Schauspieler
1,661d1a30d05f1cef398e0e98,15-2011.00,short,"{'code': '15-2011.00', 'title': 'Actuaries', '...","{'task': [{'id': 3500, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.4.a', 'related': 'htt...","{'element': [{'id': '2.A.2.a', 'related': 'htt...",...,"{'value': 4, 'title': 'Job Zone Four: Consider...","{'high_point_code': 'CIE', 'element': [{'id': ...","{'element': [{'id': '1.C.7.b', 'related': 'htt...","{'element': [{'id': '1.B.2.b', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actuary.org/'...,"{'level_required': {'category': [{'name': ""Bac...",15-2011,2120,"Mathematiker, Aktuare und Statistiker"


In [3]:
skills = pd.DataFrame()
abilities = pd.DataFrame()
tasks = pd.DataFrame()
knowledge = pd.DataFrame()
work_activities = pd.DataFrame()
detailed_work_activities = pd.DataFrame()
work_context = pd.DataFrame()
job_zone = pd.DataFrame()
work_styles = pd.DataFrame()
work_values = pd.DataFrame()
education = pd.DataFrame()

In [4]:
# Assuming df["skills"] is a column of lists of dictionaries
skills = df['skills'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
skills_df = pd.json_normalize(skills)

# Drop duplicates based on 'id'
skills_df = skills_df.drop_duplicates(subset='id')

# Reset the index
skills_df = skills_df.reset_index(drop=True)

skills_df = skills_df.dropna()
skills_df = skills_df.drop(columns=["related"])

skills_df['skill_id'] = range(1, len(skills_df) + 1)

skills_df.head(3)

,id,name,description,skill_id
0,2.A.1.a,Reading Comprehension,Understanding written sentences and paragraphs...,1
1,2.A.1.d,Speaking,Talking to others to convey information effect...,2
2,2.A.1.b,Active Listening,Giving full attention to what other people are...,3


In [5]:
dbname = get_database()

collection_name = dbname["skills"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(skills_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed0fd6b5ac15377d314'), ObjectId('661e6ed0fd6b5ac15377d315'), ObjectId('661e6ed0fd6b5ac15377d316'), ObjectId('661e6ed0fd6b5ac15377d317'), ObjectId('661e6ed0fd6b5ac15377d318'), ObjectId('661e6ed0fd6b5ac15377d319'), ObjectId('661e6ed0fd6b5ac15377d31a'), ObjectId('661e6ed0fd6b5ac15377d31b'), ObjectId('661e6ed0fd6b5ac15377d31c'), ObjectId('661e6ed0fd6b5ac15377d31d'), ObjectId('661e6ed0fd6b5ac15377d31e'), ObjectId('661e6ed0fd6b5ac15377d31f'), ObjectId('661e6ed0fd6b5ac15377d320'), ObjectId('661e6ed0fd6b5ac15377d321'), ObjectId('661e6ed0fd6b5ac15377d322'), ObjectId('661e6ed0fd6b5ac15377d323'), ObjectId('661e6ed0fd6b5ac15377d324'), ObjectId('661e6ed0fd6b5ac15377d325'), ObjectId('661e6ed0fd6b5ac15377d326'), ObjectId('661e6ed0fd6b5ac15377d327'), ObjectId('661e6ed0fd6b5ac15377d328'), ObjectId('661e6ed0fd6b5ac15377d329'), ObjectId('661e6ed0fd6b5ac15377d32a'), ObjectId('661e6ed0fd6b5ac15377d32b'), ObjectId('661e6ed0fd6b5ac15377d32c'), ObjectId('661e6ed0fd6b5ac15377d3

In [6]:
# Assuming df["abilities"] is a column of lists of dictionaries
abilities = df['abilities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
abilities_df = pd.json_normalize(abilities)

# Drop duplicates based on 'id'
abilities_df = abilities_df.drop_duplicates(subset='id')

# Reset the index
abilities_df = abilities_df.reset_index(drop=True)

abilities_df = abilities_df.dropna()
abilities_df = abilities_df.drop(columns=["related"])

abilities_df['ability_id'] = range(1, len(abilities_df) + 1)

abilities_df.head(3)

,id,name,description,ability_id
0,1.A.1.a.3,Oral Expression,The ability to communicate information and ide...,1
1,1.A.1.a.1,Oral Comprehension,The ability to listen to and understand inform...,2
2,1.A.1.d.1,Memorization,The ability to remember information such as wo...,3


In [7]:
dbname = get_database()

collection_name = dbname["abilities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(abilities_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed0fd6b5ac15377d334'), ObjectId('661e6ed0fd6b5ac15377d335'), ObjectId('661e6ed0fd6b5ac15377d336'), ObjectId('661e6ed0fd6b5ac15377d337'), ObjectId('661e6ed0fd6b5ac15377d338'), ObjectId('661e6ed0fd6b5ac15377d339'), ObjectId('661e6ed0fd6b5ac15377d33a'), ObjectId('661e6ed0fd6b5ac15377d33b'), ObjectId('661e6ed0fd6b5ac15377d33c'), ObjectId('661e6ed0fd6b5ac15377d33d'), ObjectId('661e6ed0fd6b5ac15377d33e'), ObjectId('661e6ed0fd6b5ac15377d33f'), ObjectId('661e6ed0fd6b5ac15377d340'), ObjectId('661e6ed0fd6b5ac15377d341'), ObjectId('661e6ed0fd6b5ac15377d342'), ObjectId('661e6ed0fd6b5ac15377d343'), ObjectId('661e6ed0fd6b5ac15377d344'), ObjectId('661e6ed0fd6b5ac15377d345'), ObjectId('661e6ed0fd6b5ac15377d346'), ObjectId('661e6ed0fd6b5ac15377d347'), ObjectId('661e6ed0fd6b5ac15377d348'), ObjectId('661e6ed0fd6b5ac15377d349'), ObjectId('661e6ed0fd6b5ac15377d34a'), ObjectId('661e6ed0fd6b5ac15377d34b'), ObjectId('661e6ed0fd6b5ac15377d34c'), ObjectId('661e6ed0fd6b5ac15377d3

In [8]:
# Assuming df["tasks"] is a column of lists of dictionaries
tasks = df['tasks'].apply(lambda x: x['task']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
tasks_df = pd.json_normalize(tasks)

# Drop duplicates based on 'id'
tasks_df = tasks_df.drop_duplicates(subset='id')

# Reset the index
tasks_df = tasks_df.reset_index(drop=True)

tasks_df = tasks_df.dropna()
tasks_df = tasks_df.drop(columns=["green", "related"])

tasks_df['task_id'] = range(1, len(tasks_df) + 1)

tasks_df.head(3)

,id,name,task_id
0,7646,Collaborate with other actors as part of an en...,1
1,7649,"Portray and interpret roles, using speech, ges...",2
2,7644,"Work closely with directors, other actors, and...",3


In [9]:
dbname = get_database()

collection_name = dbname["tasks"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(tasks_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed1fd6b5ac15377d35f'), ObjectId('661e6ed1fd6b5ac15377d360'), ObjectId('661e6ed1fd6b5ac15377d361'), ObjectId('661e6ed1fd6b5ac15377d362'), ObjectId('661e6ed1fd6b5ac15377d363'), ObjectId('661e6ed1fd6b5ac15377d364'), ObjectId('661e6ed1fd6b5ac15377d365'), ObjectId('661e6ed1fd6b5ac15377d366'), ObjectId('661e6ed1fd6b5ac15377d367'), ObjectId('661e6ed1fd6b5ac15377d368'), ObjectId('661e6ed1fd6b5ac15377d369'), ObjectId('661e6ed1fd6b5ac15377d36a'), ObjectId('661e6ed1fd6b5ac15377d36b'), ObjectId('661e6ed1fd6b5ac15377d36c'), ObjectId('661e6ed1fd6b5ac15377d36d'), ObjectId('661e6ed1fd6b5ac15377d36e'), ObjectId('661e6ed1fd6b5ac15377d36f'), ObjectId('661e6ed1fd6b5ac15377d370'), ObjectId('661e6ed1fd6b5ac15377d371'), ObjectId('661e6ed1fd6b5ac15377d372'), ObjectId('661e6ed1fd6b5ac15377d373'), ObjectId('661e6ed1fd6b5ac15377d374'), ObjectId('661e6ed1fd6b5ac15377d375'), ObjectId('661e6ed1fd6b5ac15377d376'), ObjectId('661e6ed1fd6b5ac15377d377'), ObjectId('661e6ed1fd6b5ac15377d3

In [10]:
# Assuming df["knowledge"] is a column of lists of dictionaries
knowledge = df['knowledge'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
knowledge_df = pd.json_normalize(knowledge)

# Drop duplicates based on 'id'
knowledge_df = knowledge_df.drop_duplicates(subset='id')

# Reset the index
knowledge_df = knowledge_df.reset_index(drop=True)

knowledge_df = knowledge_df.dropna()
knowledge_df = knowledge_df.drop(columns=["related"])

knowledge_df['knowledge_id'] = range(1, len(knowledge_df) + 1)

knowledge_df.head(3)

,id,name,description,knowledge_id
0,2.C.7.c,Fine Arts,Knowledge of the theory and techniques require...,1
1,2.C.7.a,English Language,Knowledge of the structure and content of the ...,2
2,2.C.9.b,Communications and Media,"Knowledge of media production, communication, ...",3


In [11]:
dbname = get_database()

collection_name = dbname["knowledge"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(knowledge_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed6fd6b5ac15377dfb7'), ObjectId('661e6ed6fd6b5ac15377dfb8'), ObjectId('661e6ed6fd6b5ac15377dfb9'), ObjectId('661e6ed6fd6b5ac15377dfba'), ObjectId('661e6ed6fd6b5ac15377dfbb'), ObjectId('661e6ed6fd6b5ac15377dfbc'), ObjectId('661e6ed6fd6b5ac15377dfbd'), ObjectId('661e6ed6fd6b5ac15377dfbe'), ObjectId('661e6ed6fd6b5ac15377dfbf'), ObjectId('661e6ed6fd6b5ac15377dfc0'), ObjectId('661e6ed6fd6b5ac15377dfc1'), ObjectId('661e6ed6fd6b5ac15377dfc2'), ObjectId('661e6ed6fd6b5ac15377dfc3'), ObjectId('661e6ed6fd6b5ac15377dfc4'), ObjectId('661e6ed6fd6b5ac15377dfc5'), ObjectId('661e6ed6fd6b5ac15377dfc6'), ObjectId('661e6ed6fd6b5ac15377dfc7'), ObjectId('661e6ed6fd6b5ac15377dfc8'), ObjectId('661e6ed6fd6b5ac15377dfc9'), ObjectId('661e6ed6fd6b5ac15377dfca'), ObjectId('661e6ed6fd6b5ac15377dfcb'), ObjectId('661e6ed6fd6b5ac15377dfcc'), ObjectId('661e6ed6fd6b5ac15377dfcd'), ObjectId('661e6ed6fd6b5ac15377dfce'), ObjectId('661e6ed6fd6b5ac15377dfcf'), ObjectId('661e6ed6fd6b5ac15377df

In [12]:
# Assuming df["work_activities"] is a column of lists of dictionaries
work_activities = df['work_activities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_activities_df = pd.json_normalize(work_activities)

# Drop duplicates based on 'id'
work_activities_df = work_activities_df.drop_duplicates(subset='id')

# Reset the index
work_activities_df = work_activities_df.reset_index(drop=True)

work_activities_df = work_activities_df.dropna()
work_activities_df = work_activities_df.drop(columns=["related"])

work_activities_df['work_activity_id'] = range(1, len(work_activities_df) + 1)

work_activities_df.head(3)

,id,name,description,work_activity_id
0,4.A.4.a.4,Establishing and Maintaining Interpersonal Rel...,Developing constructive and cooperative workin...,1
1,4.A.2.b.2,Thinking Creatively,"Developing, designing, or creating new applica...",2
2,4.A.4.a.8,Performing for or Working Directly with the Pu...,Performing for people or dealing directly with...,3


In [13]:
dbname = get_database()

collection_name = dbname["work_activities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_activities_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed6fd6b5ac15377dfd9'), ObjectId('661e6ed6fd6b5ac15377dfda'), ObjectId('661e6ed6fd6b5ac15377dfdb'), ObjectId('661e6ed6fd6b5ac15377dfdc'), ObjectId('661e6ed6fd6b5ac15377dfdd'), ObjectId('661e6ed6fd6b5ac15377dfde'), ObjectId('661e6ed6fd6b5ac15377dfdf'), ObjectId('661e6ed6fd6b5ac15377dfe0'), ObjectId('661e6ed6fd6b5ac15377dfe1'), ObjectId('661e6ed6fd6b5ac15377dfe2'), ObjectId('661e6ed6fd6b5ac15377dfe3'), ObjectId('661e6ed6fd6b5ac15377dfe4'), ObjectId('661e6ed6fd6b5ac15377dfe5'), ObjectId('661e6ed6fd6b5ac15377dfe6'), ObjectId('661e6ed6fd6b5ac15377dfe7'), ObjectId('661e6ed6fd6b5ac15377dfe8'), ObjectId('661e6ed6fd6b5ac15377dfe9'), ObjectId('661e6ed6fd6b5ac15377dfea'), ObjectId('661e6ed6fd6b5ac15377dfeb'), ObjectId('661e6ed6fd6b5ac15377dfec'), ObjectId('661e6ed6fd6b5ac15377dfed'), ObjectId('661e6ed6fd6b5ac15377dfee'), ObjectId('661e6ed6fd6b5ac15377dfef'), ObjectId('661e6ed6fd6b5ac15377dff0'), ObjectId('661e6ed6fd6b5ac15377dff1'), ObjectId('661e6ed6fd6b5ac15377df

In [14]:
# Assuming df["detailed_work_activities"] is a column of lists of dictionaries
detailed_work_activities = df['detailed_work_activities'].apply(lambda x: x['activity']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
detailed_work_activities_df = pd.json_normalize(detailed_work_activities)

# Drop duplicates based on 'id'
detailed_work_activities_df = detailed_work_activities_df.drop_duplicates(subset='id')

# Reset the index
detailed_work_activities_df = detailed_work_activities_df.reset_index(drop=True)

detailed_work_activities_df = detailed_work_activities_df.dropna()
detailed_work_activities_df = detailed_work_activities_df.drop(columns=["related"])

detailed_work_activities_df['detailed_work_activity_id'] = range(1, len(detailed_work_activities_df) + 1)

detailed_work_activities_df.head(3)

,id,name,detailed_work_activity_id
0,4.A.4.a.8.I01.D04,Collaborate with others to prepare or perform ...,1
1,4.A.4.a.8.I01.D05,Entertain public with comedic or dramatic perf...,2
2,4.A.1.a.1.I01.D04,Study scripts to determine project requirements.,3


In [15]:
dbname = get_database()

collection_name = dbname["detailed_work_activities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(detailed_work_activities_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed6fd6b5ac15377e002'), ObjectId('661e6ed6fd6b5ac15377e003'), ObjectId('661e6ed6fd6b5ac15377e004'), ObjectId('661e6ed6fd6b5ac15377e005'), ObjectId('661e6ed6fd6b5ac15377e006'), ObjectId('661e6ed6fd6b5ac15377e007'), ObjectId('661e6ed6fd6b5ac15377e008'), ObjectId('661e6ed6fd6b5ac15377e009'), ObjectId('661e6ed6fd6b5ac15377e00a'), ObjectId('661e6ed6fd6b5ac15377e00b'), ObjectId('661e6ed6fd6b5ac15377e00c'), ObjectId('661e6ed6fd6b5ac15377e00d'), ObjectId('661e6ed6fd6b5ac15377e00e'), ObjectId('661e6ed6fd6b5ac15377e00f'), ObjectId('661e6ed6fd6b5ac15377e010'), ObjectId('661e6ed6fd6b5ac15377e011'), ObjectId('661e6ed6fd6b5ac15377e012'), ObjectId('661e6ed6fd6b5ac15377e013'), ObjectId('661e6ed6fd6b5ac15377e014'), ObjectId('661e6ed6fd6b5ac15377e015'), ObjectId('661e6ed6fd6b5ac15377e016'), ObjectId('661e6ed6fd6b5ac15377e017'), ObjectId('661e6ed6fd6b5ac15377e018'), ObjectId('661e6ed6fd6b5ac15377e019'), ObjectId('661e6ed6fd6b5ac15377e01a'), ObjectId('661e6ed6fd6b5ac15377e0

In [16]:
# Assuming df["work_context"] is a column of lists of dictionaries
work_context = df['work_context'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_context_df = pd.json_normalize(work_context)

# Drop duplicates based on 'id'
work_context_df = work_context_df.drop_duplicates(subset='id')

# Reset the index
work_context_df = work_context_df.reset_index(drop=True)

work_context_df = work_context_df.dropna()
work_context_df = work_context_df.drop(columns=["related", "response"])

work_context_df['work_context_id'] = range(1, len(work_context_df) + 1)

work_context_df.head(3)

,id,name,description,work_context_id
0,4.C.1.b.1.e,Work With Work Group or Team,How important is it to work with others in a g...,1
1,4.C.2.a.3,Physical Proximity,To what extent does this job require the worke...,2
2,4.C.1.a.4,Contact With Others,How much does this job require the worker to b...,3


In [17]:
dbname = get_database()

collection_name = dbname["work_context"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_context_df.to_dict("records"))

InsertManyResult([ObjectId('661e6ed8fd6b5ac15377e567'), ObjectId('661e6ed8fd6b5ac15377e568'), ObjectId('661e6ed8fd6b5ac15377e569'), ObjectId('661e6ed8fd6b5ac15377e56a'), ObjectId('661e6ed8fd6b5ac15377e56b'), ObjectId('661e6ed8fd6b5ac15377e56c'), ObjectId('661e6ed8fd6b5ac15377e56d'), ObjectId('661e6ed8fd6b5ac15377e56e'), ObjectId('661e6ed8fd6b5ac15377e56f'), ObjectId('661e6ed8fd6b5ac15377e570'), ObjectId('661e6ed8fd6b5ac15377e571'), ObjectId('661e6ed8fd6b5ac15377e572'), ObjectId('661e6ed8fd6b5ac15377e573'), ObjectId('661e6ed8fd6b5ac15377e574'), ObjectId('661e6ed8fd6b5ac15377e575'), ObjectId('661e6ed8fd6b5ac15377e576'), ObjectId('661e6ed8fd6b5ac15377e577'), ObjectId('661e6ed8fd6b5ac15377e578'), ObjectId('661e6ed8fd6b5ac15377e579'), ObjectId('661e6ed8fd6b5ac15377e57a'), ObjectId('661e6ed8fd6b5ac15377e57b'), ObjectId('661e6ed8fd6b5ac15377e57c'), ObjectId('661e6ed8fd6b5ac15377e57d'), ObjectId('661e6ed8fd6b5ac15377e57e'), ObjectId('661e6ed8fd6b5ac15377e57f'), ObjectId('661e6ed8fd6b5ac15377e5

In [18]:
# Assuming df["work_styles"] is a column of lists of dictionaries
work_styles = df['work_styles'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_styles_df = pd.json_normalize(work_styles)

# Drop duplicates based on 'id'
work_styles_df = work_styles_df.drop_duplicates(subset='id')

# Reset the index
work_styles_df = work_styles_df.reset_index(drop=True)

work_styles_df = work_styles_df.dropna()
work_styles_df = work_styles_df.drop(columns=["related"])

work_styles_df['work_style_id'] = range(1, len(work_styles_df) + 1)

work_styles_df.head(3)

,id,name,description,work_style_id
0,1.C.3.a,Cooperation,Job requires being pleasant with others on the...,1
1,1.C.1.b,Persistence,Job requires persistence in the face of obstac...,2
2,1.C.4.c,Adaptability/Flexibility,Job requires being open to change (positive or...,3


In [19]:
dbname = get_database()

collection_name = dbname["work_styles"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_styles_df.to_dict("records"))

InsertManyResult([ObjectId('661e6edafd6b5ac15377e59a'), ObjectId('661e6edafd6b5ac15377e59b'), ObjectId('661e6edafd6b5ac15377e59c'), ObjectId('661e6edafd6b5ac15377e59d'), ObjectId('661e6edafd6b5ac15377e59e'), ObjectId('661e6edafd6b5ac15377e59f'), ObjectId('661e6edafd6b5ac15377e5a0'), ObjectId('661e6edafd6b5ac15377e5a1'), ObjectId('661e6edafd6b5ac15377e5a2'), ObjectId('661e6edafd6b5ac15377e5a3'), ObjectId('661e6edafd6b5ac15377e5a4'), ObjectId('661e6edafd6b5ac15377e5a5'), ObjectId('661e6edafd6b5ac15377e5a6'), ObjectId('661e6edafd6b5ac15377e5a7'), ObjectId('661e6edafd6b5ac15377e5a8'), ObjectId('661e6edafd6b5ac15377e5a9')], acknowledged=True)

In [20]:
# Assuming df["work_values"] is a column of lists of dictionaries
work_values = df['work_values'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_values_df = pd.json_normalize(work_values)

# Drop duplicates based on 'id'
work_values_df = work_values_df.drop_duplicates(subset='id')

# Reset the index
work_values_df = work_values_df.reset_index(drop=True)

work_values_df = work_values_df.dropna()
work_values_df = work_values_df.drop(columns=["related"])

work_values_df['work_value_id'] = range(1, len(work_values_df) + 1)

work_values_df.head(3)

,id,name,description,work_value_id
0,1.B.2.d,Relationships,Occupations that satisfy this work value allow...,1
1,1.B.2.a,Achievement,Occupations that satisfy this work value are r...,2
2,1.B.2.f,Independence,Occupations that satisfy this work value allow...,3


In [21]:
dbname = get_database()

collection_name = dbname["work_values"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_values_df.to_dict("records"))

InsertManyResult([ObjectId('661e6edafd6b5ac15377e5ab'), ObjectId('661e6edafd6b5ac15377e5ac'), ObjectId('661e6edafd6b5ac15377e5ad'), ObjectId('661e6edafd6b5ac15377e5ae'), ObjectId('661e6edafd6b5ac15377e5af'), ObjectId('661e6edafd6b5ac15377e5b0')], acknowledged=True)

In [22]:
job_zone = pd.DataFrame(list(df["job_zone"]))

# Drop duplicates based on 'value'
job_zone = job_zone.drop_duplicates(subset='value')
job_zone = job_zone.dropna()
job_zone = job_zone.drop(columns=["svp_range"])

job_zone['job_zone_id'] = range(1, len(job_zone) + 1)

job_zone.head(2)

,value,title,education,related_experience,job_training,job_zone_examples,job_zone_id
0,2,Job Zone Two: Some Preparation Needed,These occupations usually require a high schoo...,"Some previous work-related skill, knowledge, o...",Employees in these occupations need anywhere f...,These occupations often involve using your kno...,1
1,4,Job Zone Four: Considerable Preparation Needed,Most of these occupations require a four-year ...,"A considerable amount of work-related skill, k...",Employees in these occupations usually need se...,Many of these occupations involve coordinating...,2


In [23]:
dbname = get_database()

collection_name = dbname["job_zone"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(job_zone.to_dict("records"))

InsertManyResult([ObjectId('661e6edafd6b5ac15377e5b2'), ObjectId('661e6edafd6b5ac15377e5b3'), ObjectId('661e6edafd6b5ac15377e5b4'), ObjectId('661e6edafd6b5ac15377e5b5'), ObjectId('661e6edafd6b5ac15377e5b6')], acknowledged=True)

In [24]:
# Assuming df["work_values"] is a column of lists of dictionaries
education = df['education'].apply(lambda x: x['level_required']).apply(lambda x : x["category"]).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
education_df = pd.json_normalize(education)

# Drop duplicates based on 'name'
education_df = education_df.drop_duplicates(subset='name')
education_df = education_df.drop(columns=["score.scale", "score.value", "description"])
education_df = education_df.dropna()

education_df['education_id'] = range(1, len(education_df) + 1)

education_df.head(5)

,name,education_id
0,Less than high school diploma,1
1,Bachelor's degree,2
2,High school diploma or equivalent,3
4,Post-baccalaureate certificate,4
5,Professional degree,5


In [25]:
dbname = get_database()

collection_name = dbname["education"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(education_df.to_dict("records"))

InsertManyResult([ObjectId('661e6edbfd6b5ac15377e5b8'), ObjectId('661e6edbfd6b5ac15377e5b9'), ObjectId('661e6edbfd6b5ac15377e5ba'), ObjectId('661e6edbfd6b5ac15377e5bb'), ObjectId('661e6edbfd6b5ac15377e5bc'), ObjectId('661e6edbfd6b5ac15377e5bd'), ObjectId('661e6edbfd6b5ac15377e5be'), ObjectId('661e6edbfd6b5ac15377e5bf'), ObjectId('661e6edbfd6b5ac15377e5c0'), ObjectId('661e6edbfd6b5ac15377e5c1'), ObjectId('661e6edbfd6b5ac15377e5c2'), ObjectId('661e6edbfd6b5ac15377e5c3')], acknowledged=True)